### Coursera Capstone Project for IBM Data Science Certification

In [1]:
#Libararies

import pandas as pd
import numpy as np

print('Hello Capstone Project Course!')

Hello Capstone Project Course!


##### Web Scrapping using BeautifulSoup

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
#defining the function to make the request
def get_data_from_url(url):
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data,'html5lib')
    #print(soup)
    df = pd.DataFrame()
    for row in soup.find_all('tr'):
        arr = []
        for col in row.find_all('td'):
            arr.append(col.getText())  # col.getText()--> returning string
        df = df.append(pd.DataFrame(arr).T)
    return df

In [4]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [5]:
df = get_data_from_url(URL)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,M1A\n,Not assigned\n,Not assigned\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,M2A\n,Not assigned\n,Not assigned\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,M3A\n,North York\n,Parkwoods\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,M4A\n,North York\n,Victoria Village\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.columns = [str(n) for n in range(len(df.columns))] #for getting required columns only
df = df[['0','1','2']].replace(to_replace=r'\n',value='',regex=True)
df.reset_index(drop=True, inplace=True)
df.columns = ['Postal Code','Borough','Neighbourhood']

In [7]:
pd.set_option('display.max_rows', None)  # for validating data if possible

In [8]:
df.tail(5)

,Postal Code,Borough,Neighbourhood
179,M9Z,Not assigned,Not assigned
180,,,NaN
181,NLNSPENBQCONMBSKABBCNU/NTYTABCEGHJKLMNPRSTVXY,NL,NS
182,NL,NS,PE
183,A,B,C


In [9]:
df = df.iloc[range(df.shape[0]-4)]  #Removing last 4 outlayers

In [10]:
df = df[df['Borough']!='Not assigned']  #Removed not assigned values

In [11]:
df[df['Neighbourhood']=='Not assigned']  # checking weather 'Not assigned' exist in Neighbourhood
df.reset_index(drop=True, inplace=True)

In [12]:
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Webdata Extracted and ready for further processing

In [13]:
############ Not worked############
# import geocoder # import geocoder

# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]
# print(latitude,longitude)

In [14]:
latlon  =  pd.read_csv('Geospatial_Coordinates.csv',delimiter=',')
df  = pd.merge(df, latlon, left_on='Postal Code', right_on='Postal Code')

In [15]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
